In [7]:
import cv2 as cv
import collections as col
import os.path
from typing import List
import os
import datetime
#import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow.keras.layers import Normalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

In [20]:



def load_images_from_folder(images_FP7):
    images = []
    list_lw = []
    for bild in os.listdir(images_FP7):
    
        #print(bild)
        name = bild.split('_')
        lw_in_name = name[3].split('.')
        lenkwinkel = int(lw_in_name[0])
        # print(name)
        # print(lenkwinkel)
        img = cv.imread(os.path.join(images_FP7,bild))
        #image_lw = [img, lenkwinkel]
        if img is not None:
            images.append(img)
            list_lw.append(lenkwinkel)
            
    return images, list_lw

    #Lenkwinkel_Dict:
    #path_dir: str = r"/home/imm-81/c2c/camp2code-project_phase_2/Code/images_FP7"

    #content_dir: List[str] = os.listdir(path_dir)
#df = pd.DataFrame
X, y = load_images_from_folder('images_FP7')
X = np.array(X)
y = np.array(y).astype('float')
print(X.shape)
print(y.shape)

# Bild_list_array = load_images_from_folder('images_FP7')
# print(len(Bild_list_array))
# bild_act = Bild_list_array[0]
# print(bild_act[1])
# print(len(bild_act[0]))
# print(bild_act[0].shape)
# cv.imshow('test', bild_act[0])
# cv.waitKey(0)
# cv.destroyAllWindows()





(157, 242, 594, 3)
(157,)


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
# print(X_train)

In [8]:
# Keras Sequntial API

from tensorflow.keras.layers import Conv2D, Dropout, Flatten
model = tf.keras.Sequential(name='Team_Israel')
# Convolution Layers
# elu: Expenential Linear Unit, similar to leaky Relu
model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(242, 594, 3), activation='elu'))
model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu'))
model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu'))
model.add(Conv2D(64, (3, 3), activation='elu'))
model.add(Dropout(0.2)) # more robustness
model.add(Conv2D(64, (3, 3), activation='elu'))
# Fully Connected Layers
model.add(Flatten())
model.add(Dropout(0.2)) # more robustness
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
# Output Layer: turning angle
model.add(Dense(1)) 

In [9]:
model.summary()

Model: "Team_Israel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 119, 295, 24)      1824      
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 58, 146, 36)       21636     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 27, 71, 48)        43248     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 25, 69, 64)        27712     
_________________________________________________________________
dropout_4 (Dropout)          (None, 25, 69, 64)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 23, 67, 64)        36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 98624)             

In [28]:
# Kompilierung des Modells. Ein nicht kompiliertes Modell kann nicht trainiert werden!
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=0.05), # Festlegen eines Opimizers und der Schrittweite
    loss='mse')

In [29]:
# Durchführung des Trainings mittels der Methode fit am kompilierten Modell.
# Die Methode gibt ein Objekt zurück, welches den Trainingsverlauf dokumentiert.
history = model.fit(
    X_train,                 # Trainingsmerkmale
    y_train,                 # Trainingszielmerkmal
    batch_size=32,           # Anzahl der verwendeten Trainingsdatensätze pro Trainingsschritt (Gradientenabstieg)
    epochs=100,              # Eine Epoche ist beendet wenn alle Traingsdaten verwendet wurden.
    verbose=1,               # Unterdrückt Ausgabe während des Trainings
    validation_split = 0.2,  # Es wird ein Validierungset verwendet. Dieses ist in jeder Epoche das Selbe.
    )

print('Type des Rückgabeobjektes',type(history))
print('Type des Attribute "history" Rückgabeobjektes',type(history))
print('Keys des Dictionaries:',history.history.keys())

Epoch 1/100
2/4 [==============>...............] - ETA: 50s - loss: 9766.0435  

KeyboardInterrupt: 

In [3]:
import tensorflow as tf
inputs = tf.keras.Input(shape=X_train[0].shape)           # Erzeugt eine Eingabeschicht für alle unabhängigen Merkmale
print('-Eingabeschicht:\n',inputs)                        # -> inputs ist eine KerasTensor

# STANDARDISIERUNGSSCHICHT
normalized_inputs = normalizer(inputs)                    # Verküpfung der Eingabeschicht mit der Standardisierungsschicht. Die Eingabeschicht dient als Input der Standardisierungschicht.
print('\n-Objekt Standardisierungschicht:',normalizer)    # -> der normalizer ist ein Object - kein KeraTensor
print('-Standardisierungsschicht:\n',normalized_inputs)   # -> normalized_input ist ein KerasTensor

# AUSGABESCHICHT
output_layer = tf.keras.layers.Dense(                     # Erzeugt eine Dense-Schicht mit einer Zelle. 
                    units=1,                              # Die Schicht enthält nur eine Zelle, da die Regression nur für eine Zeilvariable hat.                          
                    activation=None,                      # Es wird keine Aktivierungsfunktion verwendet (Lineare Regression).
                    use_bias=True,                        # Ein Bias wird verwendet. Absolutglied in der Regression.
                    kernel_initializer="glorot_uniform",  # Xavier-Initialisierung der Gewichte
                    bias_initializer="zeros")             # Bias-Werte werden mit Null initialisiert
outputs =output_layer(normalized_inputs)                  # Die standardisierten Inputs werden der Dense-Schicht, als der Inputs, übergeben.
print('-Ausgabeschicht:\n',outputs)
# Anmerkung: "Dense" bedeutet, dass alle Zellen des Inputs mit allen Zellen der Schicht durch ein Gewicht verbunden sind.


# Erstellung eines Keras Modells durch Definition der Eingabe und der Ausgabe (name ist optional)
linear_model_FuncAPI = tf.keras.Model(inputs = inputs, outputs = outputs, name = 'LinearRegressionFuncAPI')

# Ein Kerasmodell verfügt über die Methode summary, welche den Aufbau des Netzes zusammenfässt.
print('\nOBJEKTTYPE DES MODELLS:',type(linear_model_FuncAPI))
print('SUMMARY:')
linear_model_FuncAPI.summary()

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 44 from C header, got 40 from PyObject